In [ ]:
!pip install nlpaug
!pip install transformers
!pip install swifter

     |████████████████████████████████| 389kB 5.1MB/s 
     |████████████████████████████████| 1.3MB 5.1MB/s 
     |████████████████████████████████| 2.9MB 13.4MB/s 
     |████████████████████████████████| 1.1MB 30.6MB/s 
     |████████████████████████████████| 890kB 38.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=80f97195d988a1dcfc950add295d345a10cf6bd47a6f5acb0d84803795d69984
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 634kB 3.8MB/s 
     |████████████████████████████████| 471kB 19.3MB/s 
     |████████████████████████████████| 542kB 16.6MB/s 
     |████████████████████████████████| 92kB 9.5MB/s 
     |████████████████████████████████| 23.1MB 53.7MB/s 
     |████████████████████████████████| 17.2MB 370kB/s 
     |████████████████████████████████| 133kB 46.4MB/s 
     |████████████████████████████

In [ ]:
import itertools
from typing import List, Dict, Union

import nlpaug.augmenter.word as naw
import pandas as pd
import swifter

In [ ]:
ROOT_PATH = "/content/drive/My Drive/Magíster 2020"

In [ ]:
aurora_train = pd.read_json(f"{ROOT_PATH}/Ordenado/new_train_test_data/aurora_train_70.json", orient="records")
aurora_test = pd.read_json(f"{ROOT_PATH}/Ordenado/new_train_test_data/aurora_test_30.json", orient="records")

In [ ]:
def calculate_augmentation(max_count: int, current_count: int) -> int:
    return (max_count - current_count) // current_count

In [ ]:
AURORA_MAX = aurora_train["first_sdg"].value_counts().max()
AURORA_MAX

1185

In [ ]:
def augment_abstract(row: pd.Series, current_value_counts: pd.Series) -> List[Dict[str, Union[str, int]]]:
    aug = naw.ContextualWordEmbsAug(model_path="distilbert-base-cased", action="substitute")
    sdg = row["first_sdg"]
    title = row["title"]
    current_count = current_value_counts[row["first_sdg"]]
    if current_count == AURORA_MAX:
        return []
    new_samples = calculate_augmentation(AURORA_MAX, current_count)
    augmented_samples = aug.augment(row["clean_abstract"], n=new_samples)
    augmented_samples = [{"title": title, "first_sdg": sdg, "clean_abstract": sample} for sample in augmented_samples]
    return augmented_samples

In [ ]:
aurora_value_counts = aurora_train["first_sdg"].value_counts()
aurora_train["augmented"] = aurora_train.apply(lambda row: augment_abstract(row, aurora_value_counts), axis=1)

In [ ]:
augmented_flat = list(itertools.chain(*aurora_train["augmented"].tolist()))
augmented = pd.DataFrame(augmented_flat)

In [ ]:
del aurora_train["augmented"]
aurora_train = pd.concat([aurora_train, augmented], ignore_index=True)

In [ ]:
aurora_train.to_json(f"{ROOT_PATH}/Ordenado/augmented_with_aurora/2020-09-12_aurora_augmented.json", orient="records")